In [20]:
# Import libraries
from bs4 import BeautifulSoup
import io
import keras
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pdfplumber
import requests
import re
import time
import string

from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
#### Custom functions ####
# Remove proper nouns
def remove_proper(data):
    tagged_sentence = nltk.tag.pos_tag(data.split())
    edited_sentence = [word for word, tag in tagged_sentence if tag != "NNP" and tag != "NNPS"]
    edited_sentence = ' '.join(edited_sentence)
    return edited_sentence

# Remove non-English words
def remove_nonEnglish(data):
    words = set(nltk.corpus.words.words())
    original_sentence = data.split()
    edited_sentence = [word for word in original_sentence if word in words]
    edited_sentence = ' '.join(edited_sentence)
    return edited_sentence

# Remove stop words
def remove_stopWords(data):
    stop_words = set(stopwords.words('english'))
    original_sentence = data.split()
    edited_sentence = [word for word in original_sentence if word not in stop_words]
    edited_sentence = ' '.join(edited_sentence)
    return edited_sentence

In [29]:
# Import the summary CSV file which contains species and review information
my_sp = pd.read_csv("../data/summary_table.csv")
my_sp.head(10)

my_sp = my_sp[my_sp["allReviewURLs"].notna()]

In [35]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome()

for idx, row in my_sp.iterrows():

    # Split the string of URLs to a list of each URL
    my_urls = row["allReviewURLs"].split(";")

    # Iterate over the list of URLS
    all_outputs = []
    for url in my_urls:
        # Save any natural language to a list of sentences.
        output = []

        # Confirm that the list element is actually a URL to a PDF file, otherwise skip it
        if "pdf" in url:
            request = requests.get(url)
            try:
                text = ""
                npages = len(reader.pages)
                f = io.BytesIO(request.content)
                reader = pdfplumber.open(f)

                for j in range(npages):
                    page = reader.pages[j]
                    try:
                        text += page.extract_text().apply(lambda x: remove_nonEnglish(x)).apply(lambda x: remove_proper(x)).apply(lambda x: re.sub(r'[0-9]','', x)).apply(lambda x: x.lower())
                    except:
                        pass
            except:
                pass

            output.append(text)
            reader.close()

        # Otherwise, skip that URL    
        else:
            output = []
    
        all_outputs.append(output)

        # Update the dataframe with text from the PDF file (or blank if not a PDF file)
        my_sp.at[idx, 'PDFText'] = all_outputs